# Image Processing for CZI Files

## Splitting the combined .czi file into separate channels

### List of Tasks

1. Read CZI file from a given folder.
2. Read metadata for each file and based on channel number split into separate channels.
3. Save each channel as a separate tif/tiff file with correct channel name without permission errors.

In [2]:
import glob
import os
import re
import shutil
import time

import imageio
import nrrd
import numpy as np
import SimpleITK as sitk
import tifffile as tiff
from aicspylibczi import CziFile
from scipy import ndimage
from skimage import exposure, io

### Enter the exact/full path of the folder where images are located.

In [3]:
c_path = r"C:\Users\keshavgubbi\Desktop\LinesReg\new1020"
original_path = c_path + r"\original"

In [4]:
if not os.path.exists(original_path):
    print(f"Creating {original_path}")
    os.makedirs(original_path, exist_ok=True)

In [5]:
ref_ch_num = int(input("Enter Reference Channel Number:"))

Enter Reference Channel Number: 1


In [7]:
start = time.time()
for file in os.listdir(c_path):
    if file.endswith(".czi"):
        print(file)
        name, ext = file.split(".")

        czi = CziFile(os.path.join(c_path, file))
        max_channels = range(*czi.dims_shape()[0]["C"])
        max_slices = range(*czi.dims_shape()[0]["Z"])
        for ch_num in max_channels:
            image_list = []
            for z_plane in max_slices:
                print("file:", file, ", ch_num:", ch_num, ", z_plane:", z_plane)
                imgarray, shp = czi.read_image(B=0, S=0, C=ch_num, T=0, Z=z_plane)
                image_list.append(np.squeeze(imgarray))

                if ref_ch_num == ch_num:
                    # with tiff.FileHandle(file, mode="rb") as fh:
                    with tiff.TiffWriter(
                        os.path.join(original_path, f"{name}_ref.tif"), imagej=True
                    ) as tifw:
                        print(f"Adding slice {z_plane} to {name}_ref.tif")
                        tifw.write(np.stack(image_list).astype("uint8"))
                else:
                    with tiff.TiffWriter(
                        os.path.join(original_path, f"{name}_sig.tif"), imagej=True
                    ) as tifw:
                        print(f"Adding slice {z_plane} to {name}_sig.tif")
                        tifw.write(np.stack(image_list).astype("uint8"))

end = time.time()
t = end - start
print(f"Totally took {t}s to split channels!")

s1020tEt_1_HuClyntagRFP.czi
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 0
Adding slice 0 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 1
Adding slice 1 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 2
Adding slice 2 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 3
Adding slice 3 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 4
Adding slice 4 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 5
Adding slice 5 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 6
Adding slice 6 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 7
Adding slice 7 to s1020tEt_1_HuClyntagRFP_sig.tif
file: s1020tEt_1_HuClyntagRFP.czi , ch_num: 0 , z_plane: 8
Adding slice 8 to s1020tEt_1_HuClyntagRFP

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\keshavgubbi\\Desktop\\LinesReg\\new1020\\original\\s1020tEt_1_HuClyntagRFP_sig.tif'

## Image Processing the tif files

In [8]:
line_name = input("Enter Line_name:")
tag_name = input("Enter tag name or name of the stain used:")

line_path = original_path
processed_path = line_path + "/processed/"
processed_for_average_path = line_path + "/processed_for_average/"

Enter Line_name: s1020Et
Enter tag name or name of the stain used: HUClyntagRFP


In [9]:
def image_details(f):
    print("********Image Details**************")
    img = io.imread(f)
    print(type(img))
    print("Dtype:", img.dtype)
    print("Shape:", img.shape)
    return img


def convert_to_8bit(f):
    print("********Checking Image Type********")
    # img = io.imread(f)
    dtype = f.dtype
    print(dtype)
    if dtype != "uint8":
        f.astype("uint8", copy=False)
        print(dtype)
    return f.astype("uint8")


def read_voxel_size(f):
    # print("********Reading Voxel Size********")
    im = sitk.ReadImage(f)
    width, height = im.GetSpacing()
    # print('Current voxel size:', width, height)
    return width, height


def ce(f):
    # i = img_as_float(io.imread(f)).astype(np.float64)
    logarithmic_corrected = exposure.adjust_log(f, 3)
    return logarithmic_corrected


def _rotate(src, angle):
    # angle in degrees
    rotated_matrix = ndimage.rotate(src, angle=angle, reshape=False)
    return rotated_matrix


def tiff_unstackAndrestack(f):
    """
    :param f: tiff file
    :return: rotated_image_stack
    #1. Iterate through each file as a tiff file.
    #2. split into individual pages //Unstacking
    #3. rotate each page and save the rotated_page into a new list
    #4. restack each array from the list
    """
    with tiff.TiffFile(f, mode="r+b") as tif:
        print(f" Processing {tif} for rotation...")
        for page in tif.pages:
            rotated_page = _rotate(page.asarray(), theta)
            rotated_page_list.append(rotated_page)
            rotated_image_stack = np.stack(rotated_page_list)
    return rotated_image_stack.astype("uint8")

In [10]:
def split_and_rename(f):
    filename, exte = f.split(".")
    _first, _last = filename.split("_", 1)
    fishnum, tag = _last.split("_", 1)
    # print(_first, fishnum, ext)
    return _first, fishnum

In [11]:
for item in os.listdir(line_path):
    if item.endswith(".tif"):

        # ****Contrast Enhancement, 8bit conversion, fixing voxel Depth******#
        g = tiff.imread(os.path.join(line_path, item))
        print(f"The file {item} has dimensions : {g.shape} and is of type: {g.dtype} ")
        CE_image = ce(g)
        print(f"Creating file {item} ...")
        with tiff.TiffWriter(os.path.join(line_path, item), imagej=True) as tifw:
            tifw.write(
                CE_image.astype("uint8"),
                metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},
            )

#         #***********Rotation*********************************#
#         print(f'Image stack to be rotated: {item}')
#         theta = float(input('Enter the angle by which image to be rotated:'))
#         rotated_page_list = []
#         rotated_image = tiff_unstackAndrestack(os.path.join(line_path, item))
#         print(f'Creating Rotated Image: rotated_{item}')
#         with tiff.TiffWriter(os.path.join(line_path, f"{item}"), imagej=True) as tifw:
#             tifw.write(rotated_image.astype('uint8'), metadata={'spacing': 1.0, 'unit': 'um', 'axes': 'ZYX'})
#         print('*********************************************************')

The file s1020tEt_1_HuClyntagRFP_ref.tif has dimensions : (368, 1946, 1024) and is of type: uint8 
Creating file s1020tEt_1_HuClyntagRFP_ref.tif ...
The file s1020tEt_1_HuClyntagRFP_sig.tif has dimensions : (368, 1946, 1024) and is of type: uint8 
Creating file s1020tEt_1_HuClyntagRFP_sig.tif ...
The file s1020tEt_2_HuClyntagRFP_ref.tif has dimensions : (409, 1946, 1024) and is of type: uint8 
Creating file s1020tEt_2_HuClyntagRFP_ref.tif ...
The file s1020tEt_2_HuClyntagRFP_sig.tif has dimensions : (409, 1946, 1024) and is of type: uint8 
Creating file s1020tEt_2_HuClyntagRFP_sig.tif ...


In [12]:
# **********Final Saving of Images to respective folders********
print("Final Saving of Images to respective folders!")
if not os.path.exists(processed_path):
    print(f"Creating {processed_path}")
    os.makedirs(processed_path, exist_ok=True)

if not os.path.exists(processed_for_average_path):
    print(f"Creating {processed_for_average_path}")
    os.makedirs(processed_for_average_path, exist_ok=True)

Final Saving of Images to respective folders!


In [13]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("ref", str(item)):
        print(f"Image stack to be saved: {item}")
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name, fn = split_and_rename(item)

        print(f"Creating {name}_{fn}_{tag_name}.nrrd")
        nrrd.write(
            os.path.join(processed_path, f"{name}_{fn}_{tag_name}.nrrd"),
            readdata,
            index_order="C",
        )

        print(f"Creating {name}_{fn}_{tag_name}.tif")
        with tiff.TiffWriter(
            os.path.join(processed_for_average_path, f"{name}_{fn}_{tag_name}.tif"),
            imagej=True,
        ) as tifw:
            tifw.write(
                readdata.astype("uint8"),
                metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},
            )

Image stack to be saved: s1020tEt_1_HuClyntagRFP_ref.tif
Creating s1020tEt_1_HUClyntagRFP.nrrd
Creating s1020tEt_1_HUClyntagRFP.tif
Image stack to be saved: s1020tEt_2_HuClyntagRFP_ref.tif
Creating s1020tEt_2_HUClyntagRFP.nrrd
Creating s1020tEt_2_HUClyntagRFP.tif


In [14]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("sig", str(item)):
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name = split_and_rename(item)

        print(f"Creating {name}_{fn}_GFP.nrrd")
        nrrd.write(
            os.path.join(processed_path, f"{name}_{fn}_GFP.nrrd"),
            readdata,
            index_order="C",
        )

        print(f"Creating {name}_{fn}_GFP.tif")
        with tiff.TiffWriter(
            os.path.join(processed_for_average_path, f"{name}_{fn}_GFP.tif"),
            imagej=True,
        ) as tifw:
            tifw.write(
                readdata.astype("uint8"),
                metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},
            )

end = time.time()
print("total Execution Time:", end - start, "s")

Creating ('s1020tEt', '1')_2_GFP.nrrd
Creating ('s1020tEt', '1')_2_GFP.tif
Creating ('s1020tEt', '2')_2_GFP.nrrd
Creating ('s1020tEt', '2')_2_GFP.tif
total Execution Time: 32162.10923266411 s
